In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import datetime as dt, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,ShuffleSplit
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc
from IPython.display import display, HTML
import functions as f

#modules for specific application
import random

#accessing aws data
import sys
sys.path.append('./utils')
from db_utils import DBUtil 

%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [31]:
#reload functions
reload(f);

In [2]:
#connect to aws
dbu = DBUtil("komodo_redshift","./databases/database.conf")

In [3]:
%%time
query = """
select * from jjill.jjill_master_data where is_emailable_ind='Y' limit 1000000;
"""
df = dbu.get_df_from_query(query)

CPU times: user 39.6 s, sys: 1min 57s, total: 2min 36s
Wall time: 6min 34s


In [4]:
%%time
#want customers with orders between may-july
query = """
select * from jjill.jjill_master_data 
where is_emailable_ind='Y' and
department_name in ('Woven Shirts','Knit Tops','Pants','Dresses') and
order_date between '2017-01-01' and '2017-05-01'
limit 1000000;
"""
df1 = dbu.get_df_from_query(query)

CPU times: user 41.7 s, sys: 2min 22s, total: 3min 4s
Wall time: 8min 31s


In [4]:
"filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'"

In [5]:
"""%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)"""

CPU times: user 4.5 ms, sys: 8.16 ms, total: 12.7 ms
Wall time: 37.3 ms


In [ ]:
%%time
#When I can't use AWS
#df = chunks.get_chunk(1000000)
#df['ILINK'] = df.ILINK.apply(lambda x: f.convertIlink(x))
#df = df.loc[df.IS_EMAILABLE_IND=='Y']
#df['ORDER_DATE'] = df.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
#df.columns = map(str.upper, df.columns)

In [108]:
#df1 =  f.getDF(df,'2017-1-1','2017-5-1')

In [5]:
df1.columns = map(str.upper, df1.columns)
df.columns = map(str.upper, df.columns)

In [6]:
%%time
print 'Number of unique customers:', len(df1.ILINK.unique().tolist())
ilinks = df1.ILINK.unique().tolist()

Number of unique customers: 182695
CPU times: user 1.27 s, sys: 7.01 s, total: 8.27 s
Wall time: 17.4 s


# Example Customers

In [105]:
#randomly select customers to use for example
customers = []
for i in range(100):
    customers.append(random.choice(ilinks))

In [7]:
#customers = ['0000056526']
customers=ilinks[:5000]

# Build Customer Theta Value

In [8]:
%%time
#Theta Value based on start-end timeframe
UserTheta = f.getCustTheta(customers,df1)

CPU times: user 19min 40s, sys: 45.1 s, total: 20min 25s
Wall time: 21min 11s


In [9]:
%%time
DFEXCUST = df.loc[df.ILINK.isin(customers)]
DFEXCUST.reset_index(drop=True,inplace=True)

CPU times: user 2.26 s, sys: 13.1 s, total: 15.4 s
Wall time: 28.7 s


In [10]:
%%time
#get indiv month datasets
aprildf = f.getDF(DFEXCUST,'2017-4-1','2017-5-1')
maydf = f.getDF(DFEXCUST,'2017-5-1','2017-6-1')
junedf = f.getDF(DFEXCUST,'2017-6-1','2017-7-1')
julydf = f.getDF(DFEXCUST,'2017-7-1','2017-8-1')
augdf = f.getDF(DFEXCUST,'2017-8-1','2017-9-1')
#septdf = f.getDF(DFEXCUST,'2017-9-1','2017-10-1')
#octdf = f.getDF(DFEXCUST,'2017-10-1','2017-11-1')
#novdf = f.getDF(DFEXCUST,'2017-11-1','2017-12-1')
#decdf = f.getDF(DFEXCUST,'2017-12-1','2018-1-1')

CPU times: user 4.09 s, sys: 8.18 s, total: 12.3 s
Wall time: 15.4 s


In [11]:
def checkBuy(N):
    if N > 0:
        return 1.0
    else:
        return 0.0

In [13]:
%%time
columns = ['Ilink','N_ws','S_ws','D_ws','R_ws',
           'N_kt','S_kt','D_kt','R_kt',
           'N_d','S_d','D_d','R_d',
           'N_p','S_p','D_p','R_p']
cols = ['Ilink','month','year',
           'N_ws','S_ws','D_ws','R_ws','B_ws',
           'N_kt','S_kt','D_kt','R_kt', 'B_kt',
           'N_d','S_d','D_d','R_d','B_d',
           'N_p','S_p','D_p','R_p','B_p']
table = pd.DataFrame([],columns=cols)
for ilink in customers:
    try:
        temp = f.getTableRating([ilink],aprildf,columns,UserTheta)
        temp.insert(1,'month',value=5)
        temp.insert(2,'year',value=2017)
        temp.insert(7,'B_ws',value=checkBuy(temp.N_ws.values[0]))
        temp.insert(12,'B_kt',value=checkBuy(temp.N_kt.values[0]))
        temp.insert(17,'B_d',value=checkBuy(temp.N_d.values[0]))
        temp.insert(22,'B_p',value=checkBuy(temp.N_p.values[0]))
        table = pd.concat([table,temp])
    except:
        pass
    
    try:
        temp = f.getTableRating([ilink],maydf,columns,UserTheta)
        temp.insert(1,'month',value=6)
        temp.insert(2,'year',value=2017)
        temp.insert(7,'B_ws',value=checkBuy(temp.N_ws.values[0]))
        temp.insert(12,'B_kt',value=checkBuy(temp.N_kt.values[0]))
        temp.insert(17,'B_d',value=checkBuy(temp.N_d.values[0]))
        temp.insert(22,'B_p',value=checkBuy(temp.N_p.values[0]))
        table = pd.concat([table,temp])
    except:
        pass
    try:
        temp = f.getTableRating([ilink],junedf,columns,UserTheta)
        temp.insert(1,'month',value=7)
        temp.insert(2,'year',value=2017)
        temp.insert(7,'B_ws',value=checkBuy(temp.N_ws.values[0]))
        temp.insert(12,'B_kt',value=checkBuy(temp.N_kt.values[0]))
        temp.insert(17,'B_d',value=checkBuy(temp.N_d.values[0]))
        temp.insert(22,'B_p',value=checkBuy(temp.N_p.values[0]))
        table = pd.concat([table,temp])
    except:
        pass
    
    try:
        temp = f.getTableRating([ilink],julydf,columns,UserTheta)
        temp.insert(1,'month',value=8)
        temp.insert(2,'year',value=2017)
        temp.insert(7,'B_ws',value=checkBuy(temp.N_ws.values[0]))
        temp.insert(12,'B_kt',value=checkBuy(temp.N_kt.values[0]))
        temp.insert(17,'B_d',value=checkBuy(temp.N_d.values[0]))
        temp.insert(22,'B_p',value=checkBuy(temp.N_p.values[0]))
        table = pd.concat([table,temp])
    except:
        pass
    try:
        temp = f.getTableRating([ilink],augdf,columns,UserTheta)
        temp.insert(1,'month',value=9)
        temp.insert(2,'year',value=2017)
        temp.insert(7,'B_ws',value=checkBuy(temp.N_ws.values[0]))
        temp.insert(12,'B_kt',value=checkBuy(temp.N_kt.values[0]))
        temp.insert(17,'B_d',value=checkBuy(temp.N_d.values[0]))
        temp.insert(22,'B_p',value=checkBuy(temp.N_p.values[0]))
        table = pd.concat([table,temp])
    except:
        pass

table = table[(table[cols[3:]].T != 0).any()]

CPU times: user 26min 43s, sys: 47.5 s, total: 27min 30s
Wall time: 28min 11s


In [37]:
#cust = '0000076810'
#display(f.getTableRatingV2([cust],maydf,columns))
#display(f.getTableRatingV2([cust],junedf,columns))
#display(f.getTableRatingV2([cust],julydf,columns))
#display(f.getTableRatingV2([cust],augdf,columns))

In [15]:
table.head()

,Ilink,month,year,N_ws,S_ws,D_ws,R_ws,B_ws,N_kt,S_kt,D_kt,R_kt,B_kt,N_d,S_d,D_d,R_d,B_d,N_p,S_p,D_p,R_p,B_p
0,0000000004,6,2017,0,0.00,0.00,0.00,0.0,5,154.57,83.40,0.00,1.0,0,0.0,0.0,0.0,0.0,4,184.58,93.40,0.00,1.0
0,0000000049,5,2017,0,0.00,0.00,0.00,0.0,0,0.00,0.00,0.00,0.0,0,0.0,0.0,0.0,0.0,1,62.30,26.70,0.53,1.0
0,0000000049,6,2017,0,0.00,0.00,0.00,0.0,0,0.00,0.00,0.00,0.0,0,0.0,0.0,0.0,0.0,1,0.00,0.00,0.50,1.0
0,0000000088,5,2017,0,0.00,0.00,0.00,0.0,1,30.80,13.20,0.53,1.0,0,0.0,0.0,0.0,0.0,2,124.60,53.40,0.53,1.0
0,0000000118,5,2017,1,52.53,26.47,0.11,1.0,5,139.63,70.37,0.61,1.0,0,0.0,0.0,0.0,0.0,5,262.65,132.35,0.54,1.0


# Logistic Regression Ws vs all

In [16]:
columns_X = ['month','R_ws']
X = table[columns_X]
y = table['B_ws']

In [1]:
"""%%time
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)

#Using Training data, to make validation set. 60-20-20 (training, valid, test)
cv = ShuffleSplit(n_splits=50,test_size=.25)

#Logisitc Regression
pipe = Pipeline([['sc',StandardScaler()],
                 ['lr',LogisticRegression(penalty='l2',random_state=42)]])
params = [{'lr__C': [.001,.01,1.0,10,100]}]
grid = GridSearchCV(estimator=pipe,
                    param_grid=params,
                    scoring='roc_auc',
                    cv = cv,
                    n_jobs=-1)
                    
grid.fit(X_train,y_train)
print(grid.best_score_)
print(grid.best_params_)"""

"%%time\nX_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)\n\n#Using Training data, to make validation set. 60-20-20 (training, valid, test)\ncv = ShuffleSplit(n_splits=50,test_size=.25)\n\n#Logisitc Regression\npipe = Pipeline([['sc',StandardScaler()],\n                 ['lr',LogisticRegression(penalty='l2',random_state=42)]])\nparams = [{'lr__C': [.001,.01,1.0,10,100]}]\ngrid = GridSearchCV(estimator=pipe,\n                    param_grid=params,\n                    scoring='roc_auc',\n                    cv = cv,\n                    n_jobs=-1)\n                    \ngrid.fit(X_train,y_train)\nprint(grid.best_score_)\nprint(grid.best_params_)"